In [1]:
import pandas as pd
import keras
from keras import *
from keras.layers import *
from keras.regularizers import l2
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
import operator

Using TensorFlow backend.


In [2]:
data = pd.read_pickle('foody_data.pkl')

In [3]:
data.head()

,time,user_name,user_link,brand_name,brand_link,review_content,review_link,avg_score,location_point,space_point,quality_point,service_point,price_point
0,14/9/2017 11:45,Sunny,https://www.foody.vn/thanh-vien/taiyo,El Sol - Meat & Wine - Võ Thị Sáu,https://www.foody.vn/ho-chi-minh/el-sol-meat-w...,Quán steak hiếm hoi mà mình cực kì ưng ý từ lâ...,https://www.foody.vn/ho-chi-minh/el-sol-meat-w...,8.6,7.0,10,9,9,8.0
1,14/9/2017 11:39,Nguyên Khánh,https://www.foody.vn/thanh-vien/viola_0309_tv,Busan Korean Food - Món Hàn Quốc - Đinh Tiên H...,https://www.foody.vn/ho-chi-minh/busan-korean-...,Vị trí dễ tìm. Giữ xe rất nhiệt tình. Dắt xe v...,https://www.foody.vn/ho-chi-minh/busan-korean-...,7.4,8.0,8,6,7,8.0
2,14/9/2017 11:34,Tú Hảo,https://www.foody.vn/thanh-vien/tuhao2509,TocoToco Bubble Tea - Cộng Hòa,https://www.foody.vn/ho-chi-minh/tocotoco-bubb...,"Địa điểm quán dễ tìm. Không gian cũng rộng, có...",https://www.foody.vn/ho-chi-minh/tocotoco-bubb...,6.4,7.0,8,7,7,3.0
3,14/9/2017 11:29,Vy Nhật,https://www.foody.vn/thanh-vien/zip_zilip,Le Castella Viet Nam - Bánh Bông Lan Đài Loan ...,https://www.foody.vn/ho-chi-minh/le-castella-v...,Thấy bánh này đang hot rần rần mình cũng gọi t...,https://www.foody.vn/ho-chi-minh/le-castella-v...,7.2,7.0,6,9,6,8.0
4,14/9/2017 11:26,Tien97,https://www.foody.vn/thanh-vien/tien97,Tabletop - Boardgame & Coffee,https://www.foody.vn/ho-chi-minh/tabletop-boar...,Mỗi lần nghĩ tới boardgame thì sẽ nghĩ tới vô ...,https://www.foody.vn/ho-chi-minh/tabletop-boar...,10.0,10.0,10,10,10,10.0


In [5]:
data.shape

(174437, 13)

In [28]:
review = pd.DataFrame(columns = ['user', 'item', 'rating'])
review['user'] = data['user_link'].apply(lambda x: x.replace('https://www.foody.vn/thanh-vien/',''))
review['item'] = data['brand_link'].apply(lambda x: x.replace('https://www.foody.vn/',''))
review['rating'] = data['avg_score']
review = review.groupby(['user', 'item'], as_index=False).mean() # tinh trung binh 
review.tail()

,user,item,rating
152333,zzz_mocsachdichat_zzz,hue/tony-s-house-rong-bien-say,9.0
152334,zzzbinhyenxxx,hai-phong/secret-garden-cafe-van-cao,9.0
152335,zzzthainganzzz4197,khanh-hoa/dong-suong-banh-flan,7.8
152336,zzzthainganzzz4197,khanh-hoa/zodiac-station,7.6
152337,zzzzapplezzzz,dak-lak/thanh-phat-fastfood,4.0


In [29]:
users = review.user.unique()
items = review.item.unique()

In [30]:
user2id = {o:i for i,o in enumerate(users)}
item2id = {o:i for i,o in enumerate(items)}

id2user = {v:k for k,v in user2id.items()}
id2item = {v:k for k,v in item2id.items()}

In [31]:
review['user_id'] = review.user.apply(lambda x: user2id[x])
review['item_id'] = review.item.apply(lambda x: item2id[x])

In [32]:
user2id

{'%40luciango': 0,
 '.....': 1,
 '01202056209': 2,
 '01202193703': 3,
 '01202443444': 4,
 '01202647588': 5,
 '01202673978': 6,
 '01202751104': 7,
 '01202766385': 8,
 '01203606157': 9,
 '01203950631': 10,
 '01204119717': 11,
 '01204408417': 12,
 '01205252917': 13,
 '01205297092': 14,
 '01205613350': 15,
 '01206145203': 16,
 '01206208598': 17,
 '01207668199': 18,
 '01207676268': 19,
 '01207678438': 20,
 '01208146740': 21,
 '01212101293': 22,
 '01212104808': 23,
 '01212141459': 24,
 '01212238241': 25,
 '01212246557': 26,
 '01212280271': 27,
 '01212780667': 28,
 '01212836364': 29,
 '01212871308': 30,
 '01213617660': 31,
 '01213646229': 32,
 '01213666668': 33,
 '01214433636': 34,
 '01214466926': 35,
 '01214483248': 36,
 '01214637591': 37,
 '01214644597': 38,
 '01214985350': 39,
 '01215588069': 40,
 '01215795179': 41,
 '01215977426': 42,
 '01216517470': 43,
 '01216619245': 44,
 '01216945172': 45,
 '01216983679': 46,
 '01217131423': 47,
 '01217150185': 48,
 '01217183393': 49,
 '01217857111': 

In [ ]:

# mô hình sẽ bao gồm embedding vector của từng cặp user và item được nối lại,

#tiếp đó là feed forward net, output là review của user đối với item.


In [19]:
n_user = len(set(review['user']))
n_item = len(set(review['item']))
n_dim = 32

In [20]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return(inp, Embedding(n_in, n_out, input_length=1, embeddings_regularizer=l2(reg))(inp))

In [21]:
user_in, u = embedding_input('user_in', n_user, n_dim, 1e-4)
item_in, i = embedding_input('item_in', n_item, n_dim, 1e-4)

In [22]:
x = concatenate([u, i])
x = Flatten()(x)
x = Dense(32, activation='relu')(x)
x = Dense(1)(x)
nn = Model([user_in, item_in], x)
nn.compile(Adam(0.001), loss='mse')

In [23]:
x_train, x_test, y_train, y_test = train_test_split(review[['user_id', 'item_id']], 
                                                    review['rating'], 
                                                    test_size=0.2, 
                                                    random_state=2018)

In [24]:
nn.fit([x_train['user_id'], x_train['item_id']], y_train,  
        validation_data=([x_test['user_id'],x_test['item_id']], y_test),
        epochs=5)

Train on 121870 samples, validate on 30468 samples
Epoch 1/5
121870/121870 [==============================] - 257s 2ms/step - loss: 4.2801 - val_loss: 1.7542
Epoch 2/5
121870/121870 [==============================] - 244s 2ms/step - loss: 1.4190 - val_loss: 1.8044
Epoch 3/5
121870/121870 [==============================] - 237s 2ms/step - loss: 1.1742 - val_loss: 1.8470
Epoch 4/5
121870/121870 [==============================] - 278s 2ms/step - loss: 1.0630 - val_loss: 1.8656
Epoch 5/5
121870/121870 [==============================] - 286s 2ms/step - loss: 0.9975 - val_loss: 1.8760


In [33]:
get_item_emb = Model(user_in, u)
item_emb = np.squeeze(get_item_emb.predict(review['user_id'].values))
item_emb[user2id['1anhkha']]

array([ 0.02558733,  0.00534447,  0.0033625 , -0.04715129, -0.0526593 ,
       -0.01334179, -0.01533753, -0.03333738, -0.00560608,  0.01075543,
        0.0018787 , -0.00761598, -0.01215505,  0.05526618,  0.01969042,
       -0.01093763,  0.01246077,  0.00295817, -0.05441912, -0.02504781,
       -0.00957825,  0.05266526,  0.030159  , -0.06288474,  0.01398465,
        0.05523345, -0.03180512, -0.01308822,  0.01312848, -0.0119441 ,
        0.01695549,  0.02037304], dtype=float32)

In [37]:
def recommend(user_name, location_filter = None):
    user_id = [user2id[user_name]]
    item_id = review[review['user'] != user_name].item_id.unique()
    predict = {id2item[o]:i for o,i in zip(item_id, nn.predict([np.array(user_id*len(item_id)), item_id])[:,0])}
    sorted_predict = sorted(predict.items(), key=operator.itemgetter(1), reverse=True)
    
    if location_filter is not None:
        sorted_predict = [(k,v) for k,v in sorted_predict if k.startswith(location_filter)]
    
    return(sorted_predict[:10])

In [44]:
recommend(user_name ='1anhkha')

[('ben-tre/tra-sua-family-dong-khoi', 7.306978),
 ('hue/anh-kafe-nguyen-hue', 6.684805),
 ('quang-nam/mot-hoi-an-tra-thao-moc', 6.6514163),
 ('ha-noi/co-quynh-lau-nuong', 6.620435),
 ('hue/mee-o-banh-mi-chao', 6.6041913),
 ('hue/na-house-tiem-banh-online', 6.520536),
 ('hue/trung-bo-banh-beo-banh-nam', 6.5156264),
 ('lam-dong/thongdalat-space-cafe', 6.513657),
 ('quang-nam/reaching-out-tea-house', 6.4844065),
 ('ha-noi/highlands-coffee-vtv', 6.4756017)]